In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window
import numpy as np
from pyspark.sql.functions import *
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/10/01 13:22:23 WARN Utils: Your hostname, Zhangs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.12.19.38 instead (on interface en0)
22/10/01 13:22:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/10/01 13:22:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
#transaction
ori_transaction1 = spark.read.parquet('../data/tables/transactions_20210228_20210827_snapshot').sort('order_datetime')
ori_transaction2 = spark.read.parquet('../data/tables/transactions_20210828_20220227_snapshot').sort('order_datetime')
ori_transaction3 = spark.read.parquet('../data/tables/transactions_20220228_20220828_snapshot').sort('order_datetime')
#consumer
tbl_consumer = spark.read.option("delimiter", "|").option("header",True).csv("../data/tables/tbl_consumer.csv")
consumer_detail = spark.read.parquet('../data/tables/consumer_user_details.parquet')
#fraud
consumer_fraud = spark.read.option("header",True).csv('../data/tables/consumer_fraud_probability.csv')



In [3]:
#transaction
ori_transaction = ori_transaction1.union(ori_transaction2)
ori_transaction = ori_transaction.union(ori_transaction3)
ori_transaction = ori_transaction.drop('order_id')
ori_transaction.limit(3)

user_id,merchant_abn,dollar_value,order_datetime
1,28000487688,133.22689421562643,2021-02-28
18485,62191208634,79.13140006851712,2021-02-28
1,83690644458,30.441348317517228,2021-02-28


In [4]:
#delete outlier
wind = Window.partitionBy('merchant_abn')

q1= F.expr('percentile_approx(dollar_value, 0.25)')
q3= F.expr('percentile_approx(dollar_value, 0.75)')

testq = ori_transaction.withColumn('q1', q1.over(wind))
testq = testq.withColumn('q3', q3.over(wind))

testq = testq.withColumn('IQR', testq['q3']-testq['q1'])

In [5]:
dele_out = testq.where((testq["dollar_value"] <= testq["q1"]+1.5*testq["IQR"]) & (testq["dollar_value"] >= testq["q1"]-1.5*testq["IQR"]))
ori_transaction = dele_out.drop('q1','q3','IQR','order_id')

In [6]:
ori_transaction = ori_transaction.withColumn(
    "whether_bigorder",
    F.when(F.col('dollar_value')>=10000, 1).otherwise(0))

In [7]:
#consumer
consumer = consumer_detail.join(tbl_consumer, consumer_detail.consumer_id == tbl_consumer.consumer_id).drop(tbl_consumer.consumer_id)
consumer = consumer.select('user_id', 'postcode')
consumer.limit(3)

user_id,postcode
1,6935
2,2782
3,862


In [8]:
# fraud
consumer_fraud = consumer_fraud.select(col("user_id"),col("fraud_probability"),to_date(col("order_datetime"),"yyyy-MM-dd").alias("date"))

In [9]:
#check 2021/02/28 - 2022/08/28 data
print(consumer_fraud.count())
consumer_fraud.filter("date > '2021-02-28'")
consumer_fraud.filter("date < '2022-08-28'")
print(consumer_fraud.count())

34864
34864


In [10]:
consumer_fraud_1 = consumer_fraud.groupBy('user_id').agg(F.avg('fraud_probability').alias('average_prob_con'))
consumer_fraud_final = consumer_fraud_1.withColumn(
    "whether_fraud",
    F.when(F.col('average_prob_con')>=70, 1).otherwise(0))

In [11]:
consumer_final = consumer.join(consumer_fraud_final, consumer.user_id == consumer_fraud_final.user_id,'outer').drop(consumer_fraud_final.user_id).fillna(0).fillna(0)


In [12]:
consumer_final.write.parquet(f"../data/curated/final_consumer.parquet")


22/10/01 13:22:40 WARN MemoryManager: Total allocation exceeds 95.00% (956,301,300 bytes) of heap memory
Scaling row group sizes to 89.06% for 8 writers


In [13]:
transaction = ori_transaction.join(consumer_final, ori_transaction.user_id == consumer_final.user_id).drop(consumer_final.user_id)
transaction = transaction.sort(transaction.user_id)

transaction.write.parquet(f"../data/curated/final_transaction.parquet")


22/10/01 13:23:12 WARN MemoryManager: Total allocation exceeds 95.00% (956,301,300 bytes) of heap memory
Scaling row group sizes to 89.06% for 8 writers
22/10/01 13:23:12 WARN MemoryManager: Total allocation exceeds 95.00% (956,301,300 bytes) of heap memory
Scaling row group sizes to 89.06% for 8 writers
22/10/01 13:23:13 WARN MemoryManager: Total allocation exceeds 95.00% (956,301,300 bytes) of heap memory
Scaling row group sizes to 89.06% for 8 writers
22/10/01 13:23:15 WARN MemoryManager: Total allocation exceeds 95.00% (956,301,300 bytes) of heap memory
Scaling row group sizes to 89.06% for 8 writers


In [14]:
transaction

user_id,merchant_abn,dollar_value,order_datetime,whether_bigorder,postcode,average_prob_con,whether_fraud
1,49891706470,15.730079162841406,2022-06-21,0,6935,9.805431136520959,0
1,49891706470,6.089109666984375,2021-03-30,0,6935,9.805431136520959,0
1,49891706470,49.7937996752238,2022-05-29,0,6935,9.805431136520959,0
1,28510201393,587.4024261414739,2021-09-24,0,6935,9.805431136520959,0
1,49891706470,5.904950671999753,2021-03-04,0,6935,9.805431136520959,0
1,49891706470,29.031125253031277,2021-09-11,0,6935,9.805431136520959,0
1,49891706470,31.72414306182016,2021-11-11,0,6935,9.805431136520959,0
1,81583941068,60.355511340797705,2021-08-24,0,6935,9.805431136520959,0
1,46654841462,18.503827542784197,2022-09-03,0,6935,9.805431136520959,0
1,46007761675,100.22461641821623,2022-10-24,0,6935,9.805431136520959,0
